In [121]:
import os
import random
import numpy as np
import cv2
import json
from collections import defaultdict
import torch
from torch.utils.data import Dataset, DataLoader

In [6]:
sample_jpg = '../../../input/sample/C1_감염성피부염/CYT_D_C1_000001.jpg'
sample_json = '../../../input/sample/C1_감염성피부염/CYT_D_C1_000001.json'

image = cv2.imread(sample_jpg) # (height, width, channel)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)

(1644, 2048, 3)


In [33]:
with open(sample_json, 'r') as f:
    json_data = json.load(f)
    
labels, polygons = [], []
for anno in json_data['labelingInfo']:
    if 'polygon' in anno.keys():
        labels.append(anno['polygon']['label'])
        coord_set = anno['polygon']['location']
        coord = [[[coord_set[poly_i][f'x{coord_i+1}'], coord_set[poly_i][f'y{coord_i+1}']]
                  for coord_i in range(len(coord_set[poly_i].keys())//2)] 
                 for poly_i in range(len(coord_set))]
        polygons.append((label, coord))
print(polygons)

[('C1_감염성피부염', [[[1541, 357], [1572, 622], [1637, 633], [1788, 671], [1964, 671], [2000, 591], [2015, 529], [2024, 491], [2024, 464], [2020, 446], [1928, 422], [1835, 411], [1697, 397], [1541, 357]]])]


In [ ]:
num_points_per_poly = [len(poly) for poly in polygons]
        polygons_2d = np.array([point for poly in polygons for point in poly], dtype=np.int64)
        transformed_image, transformed_polygons_2d = self.transform(image, polygons_2d)
        transformed_polygons_2d = np.around(transformed_polygons_2d).astype(np.int64) # polygon is pts argument of cv2.fillPoly. So it must be like [np.array([x,y], dtype=np.intxx)]
        num_polygons = len(num_points_per_poly)
        transformed_polygons_3d = [transformed_polygons_2d[num_points_per_poly[i-1]:num_points_per_poly[i]] \
                                   if i!=0 else transformed_polygons_2d[:num_coord_per_polygon[0]] \
                                   for i in range(num_polygons)]

In [34]:
sample_jpg = '../../../input/sample/C2_비감염성피부염/CYT_D_C6_000001.jpg'
sample_json = '../../../input/sample/C2_비감염성피부염/CYT_D_C6_000001.json'

image = cv2.imread(sample_jpg) # (height, width, channel)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)

with open(sample_json, 'r') as f:
    json_data = json.load(f)
    
polygons = []
for anno in json_data['labelingInfo']:
    if 'polygon' in anno.keys():
        label = anno['polygon']['label']
        coord_set = anno['polygon']['location']
        coord = [[[coord_set[poly_i][f'x{coord_i+1}'], coord_set[poly_i][f'y{coord_i+1}']]
                  for coord_i in range(len(coord_set[poly_i].keys())//2)] 
                 for poly_i in range(len(coord_set))]
        polygons.append((label, coord))
print(polygons)

(1644, 2048, 3)
[('C6_비감염성피부염', [[[1213, 857], [1248, 837], [1323, 758], [1386, 710], [1430, 674], [1449, 633], [1449, 624], [1413, 632], [1345, 673], [1266, 734], [1193, 819], [1194, 857], [1213, 857]]])]


In [35]:
sample_jpg = '../../../input/sample/구진플라크 무증상/IMG_D_A7_207463.jpg'
sample_json = '../../../input/sample/구진플라크 무증상/IMG_D_A7_207463.json'

image = cv2.imread(sample_jpg) # (height, width, channel)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)

with open(sample_json, 'r') as f:
    json_data = json.load(f)
    
polygons = []
for anno in json_data['labelingInfo']:
    if 'polygon' in anno.keys():
        label = anno['polygon']['label']
        coord_set = anno['polygon']['location']
        coord = [[[coord_set[poly_i][f'x{coord_i+1}'], coord_set[poly_i][f'y{coord_i+1}']]
                  for coord_i in range(len(coord_set[poly_i].keys())//2)] 
                 for poly_i in range(len(coord_set))]
        polygons.append((label, coord))
print(polygons)

(1080, 1920, 3)
[('A7_무증상', [[[592, 611], [578, 629], [578, 686], [599, 714], [621, 721], [638, 698], [631, 624], [592, 611]]])]


In [60]:
sample_jpg = '../../../input/sample/구진플라크 유증상/IMG_D_A1_000001.jpg'
sample_json = '../../../input/sample/구진플라크 유증상/IMG_D_A1_000001.json'

label_index = {'A7_무증상' : 1,
               'A1_구진_플라크': 2}

image = cv2.imread(sample_jpg) # (height, width, channel)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)

with open(sample_json, 'r') as f:
    json_data =json.load(f)
labels, polygons = [], []
for anno in json_data['labelingInfo']:
    if 'polygon' in anno.keys(): # 한 이미지에 1개~ 32개까지 있을 수 있음
        # polygon label
        labels.append(label_index[anno['polygon']['label']])
        # polygon coordinates
        coord_set = anno['polygon']['location']
        coord = [[[coord_set[poly_i][f'x{coord_i+1}'], coord_set[poly_i][f'y{coord_i+1}']]
                  for coord_i in range(len(coord_set[poly_i].keys())//2)]  
                 for poly_i in range(len(coord_set))] # 폴리곤 수
        polygons += coord
print(labels, polygons)

(1080, 1920, 3)
[2] [[[895, 287], [1365, 289], [1333, 809], [946, 815], [896, 290], [895, 287]]]


In [63]:
"""
(폴리곤 개수, 좌표 개수, xy:2)
"""
polygons_3d

array([[[ 895.,  287.],
        [1365.,  289.],
        [1333.,  809.],
        [ 946.,  815.],
        [ 896.,  290.],
        [ 895.,  287.]]])

In [61]:
from functools import reduce
def multiply(arr):
    return reduce(lambda x, y: x * y, arr)
polygons_3d = np.array(polygons, dtype=np.float64)
polygons_2d = polygons_3d.reshape(multiply(polygons_3d.shape)//2, 2)
polygons_2d

array([[ 895.,  287.],
       [1365.,  289.],
       [1333.,  809.],
       [ 946.,  815.],
       [ 896.,  290.],
       [ 895.,  287.]])

In [70]:
num_points_per_poly = [len(poly) for poly in polygons]
polygons_2d = np.array([point for poly in polygons for point in poly], dtype=np.int32)
transformed_image, transformed_polygons_2d = image, polygons_2d
transformed_polygons_2d = np.around(transformed_polygons_2d).astype(np.int32) # polygon is pts argument of cv2.fillPoly. So it must be like [np.array([x,y], dtype=np.intxx)]
num_polygons = len(num_points_per_poly)
transformed_polygons_3d = [transformed_polygons_2d[num_points_per_poly[i-1]:num_points_per_poly[i]] \
                           if i!=0 else transformed_polygons_2d[:num_points_per_poly[0]] \
                           for i in range(num_polygons)]
transformed_polygons_3d

[array([[ 895,  287],
        [1365,  289],
        [1333,  809],
        [ 946,  815],
        [ 896,  290],
        [ 895,  287]], dtype=int32)]

In [80]:
labels

[2]

In [115]:
def get_mask(labels, polygons, h, w):
    """ get a numpy mask image. The shape is like [H, W, C] """
    
    mask = np.zeros((h, w, 3), dtype=np.int64)
    for i, l in enumerate(labels):
        target = np.zeros((h, w))
        poly = polygons[0].reshape((1,) + polygons[0].shape)
        cv2.fillPoly(target, poly, l, cv2.LINE_AA)
        mask[target==l][l] = 1 # label channel
        #mask[:,:,0][mask[:,:,l]==0] = 1 # background channel
    return mask

h, w = image.shape[:2]
mask = get_mask(labels, transformed_polygons_3d, h=h, w=w)
mask.shape

(1080, 1920, 3)

In [116]:
mask = np.zeros((h, w, 3), dtype=np.int64)
for i, l in enumerate(labels):
    target = np.zeros((h, w))
    poly = transformed_polygons_3d[0].reshape((1,) + transformed_polygons_3d[0].shape)
    cv2.fillPoly(target, poly, 1, cv2.LINE_AA)
    mask[:,:,l] += target.astype(np.int64) # label channel
    mask = (mask>0).astype(np.int64)
    mask[:,:,0][mask[:,:,l]==0] = 1 # background channel

In [117]:
print(mask.shape, target.shape)

(1080, 1920, 3) (1080, 1920)


In [120]:
print(np.max(mask[:,:,0]))

1


In [114]:
mask[:,:,l] += target.astype(np.int64)
(mask > 0).astype(np.int64)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

In [85]:
np.max(mask[:,:,2])

0

In [1]:
import os
import random
import numpy as np
import cv2
import json
from collections import defaultdict
import torch
from torch.utils.data import Dataset, DataLoader
from dataloader import PetPapulePlaqueDataset

root = '../../../input/data/피부염'

dataset = PetPapulePlaqueDataset(root, 'train', 0.2, 0.2, 0.4, 512)

In [11]:
data = dataset[250]
data

{'img_name': '../../../input/data/피부염/C1_감염성피부염/CYT_D_C1_000375.jpg',
 'input': tensor([[[0.7255, 0.7020, 0.6980,  ..., 0.8000, 0.8078, 0.8118],
          [0.7333, 0.7020, 0.7373,  ..., 0.8039, 0.8157, 0.8118],
          [0.7569, 0.7333, 0.7529,  ..., 0.8000, 0.8118, 0.8039],
          ...,
          [0.8118, 0.8118, 0.8039,  ..., 0.8196, 0.8118, 0.8196],
          [0.8078, 0.8078, 0.8000,  ..., 0.8078, 0.8157, 0.8196],
          [0.8118, 0.8118, 0.8039,  ..., 0.8196, 0.8275, 0.8235]],
 
         [[0.6392, 0.6510, 0.6627,  ..., 0.7020, 0.7294, 0.7216],
          [0.6314, 0.6353, 0.6549,  ..., 0.7176, 0.7137, 0.7137],
          [0.6706, 0.6667, 0.6706,  ..., 0.7176, 0.7059, 0.7059],
          ...,
          [0.7333, 0.7490, 0.7373,  ..., 0.7216, 0.7294, 0.7294],
          [0.7255, 0.7333, 0.7294,  ..., 0.7176, 0.7176, 0.7255],
          [0.7176, 0.7255, 0.7255,  ..., 0.6980, 0.7059, 0.7255]],
 
         [[0.6549, 0.6549, 0.6471,  ..., 0.6471, 0.6510, 0.6235],
          [0.6588, 0.6588, 

In [9]:
i = 7
batch_size=3
dataset = PetPapulePlaqueDataset('../../../input/data/피부염', 'train', 0.2, 0.2, 0.4, 512)
for j in range(len(dataset)):
    try:
        data = dataset[j]
    except:
        print(dataset.jsons[j])

In [7]:
from PIL import Image
im = Image.open(dataset.jsons[0].replace('json', 'jpg'))
print(im.verify())

None


In [10]:
for j in range(len(dataset)):
    im = Image.open(dataset.jsons[j].replace('json', 'jpg'))
    im.verify()